In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from portfolio import FractilePortfolio
from enum import Enum
from tqdm import tqdm
from results import Results   
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
import plotly.graph_objects as go

In [11]:
df = pd.read_excel("datas/Stoxx600_sectors_prices_cleen.xlsx", sheet_name="Sheet1")

In [12]:
price_df = df.set_index("Date")
returns_df = price_df.pct_change().fillna(0).reset_index()

# Initialiser la structure du DataFrame final
factor_df = pd.DataFrame()
factor_df["Date"] = np.repeat(price_df.index, len(price_df.columns))
factor_df["Ticker"] = np.tile(price_df.columns, len(price_df))


P_t_12 = price_df.shift(252)
P_t_1 = price_df.shift(21)
factor_df["Momentum"] = (P_t_1 / P_t_12 - 1).values.flatten()
factor_df.dropna(inplace=True)
all_dates = sorted(set(factor_df["Date"].tolist()))

In [13]:
portfolio = FractilePortfolio(factor_df,"Momentum", 4)

In [14]:
class FrequencyType(Enum):
    DAILY = 252 # 252 jours de trading dans une année
    WEEKLY = 52 # 52 semaines dans une année
    MONTHLY = 12 # 12 mois dans une année
    HALF_EXPOSURE = "HALF_EXPOSURE" # Exposition demie vie
    UNDESIRED_EXPOSURE = "UNDESIRED" # Exposition non désirée

In [15]:
def get_rebalancing_dates(dates : list, frequency: FrequencyType) -> list[int]:
    """
    Repère les indices correspondant aux dates de rebalancement en fonction de la fréquence donnée.
    
    Args:
        dates (list[pd.Timestamp]): Liste des dates disponibles.
        frequency (FrequencyType): Fréquence de rebalancement souhaitée.
    
    Returns:
        list[int]: Indices des dates de rebalancement.
    """
    date_series = pd.Series(dates).sort_values().reset_index(drop=True)
    if frequency == FrequencyType.MONTHLY:
        # On récupère la dernière date de chaque mois
        rebalancing_dates = date_series.groupby(date_series.dt.to_period("M")).last().tolist()
    elif frequency == FrequencyType.WEEKLY:
        # On récupère la dernière date de chaque semaine
        rebalancing_dates = date_series.groupby(date_series.dt.to_period("W")).last().tolist()
    elif frequency == FrequencyType.DAILY:
        # Toutes les dates sont des dates de rebalancement
        rebalancing_dates = date_series.tolist()
    else:
        raise ValueError("Fréquence non reconnue. Utilisez 'MONTHLY', 'WEEKLY' ou 'DAILY'.")
    
    indices = [date_series[date_series == d].index[0] for d in rebalancing_dates]

    return indices

def calculate_transaction_costs(old_weights: dict, new_weights: dict, fees: float) -> float:
    """
    Calcule les frais de transaction basés sur les changements de poids.

    Args:
        old_weights (dict): Poids des actifs avant le rebalancement (ticker -> poids).
        new_weights (dict): Poids des actifs après le rebalancement (ticker -> poids).
        fees (float): Taux des frais de transaction (par exemple, 0.0005 pour 0.05%).

    Returns:
        float: Coût total des transactions.
    """
    # Obtenir l'ensemble des tickers impliqués
    all_tickers = set(old_weights.keys()).union(set(new_weights.keys()))

    # Calculer les frais de transaction pour chaque ticker
    transaction_costs = fees * np.sum(
        np.abs(np.array([new_weights.get(t, 0) - old_weights.get(t, 0) for t in all_tickers]))
    )

    return transaction_costs

def output(strategy_name : str, stored_values : list[float], stored_weights : list[float], 
               dates : list, rebalancing_dates : list , 
               fees : float = 0, frequency_data : FrequencyType = FrequencyType.DAILY) -> Results :
        """Create the output for the strategy and its benchmark if selected
        
        Args:
            stored_values (list[float]): Value of the strategy over time
            stored_weights (list[float]): Weights of every asset in the strategy over time
            strategy_name (str) : Name of the current strategy

        Returns:
            Results: A Results object containing statistics and comparison plot for the strategy (& the benchmark if selected)
        """

        ptf_weights = pd.DataFrame(stored_weights).T
        ptf_values = pd.Series(stored_values, index=dates)
        ptf_rebalacing = pd.Series([1 if date in rebalancing_dates else 0 for date in dates], index=dates)


        results_strat = Results(ptf_values=ptf_values, ptf_weights=ptf_weights, 
                                ptf_rebalancing=ptf_rebalacing, total_fees = fees,
                                strategy_name=strategy_name, data_frequency=frequency_data)
        
        results_strat.get_statistics()
        results_strat.create_plots()

        return results_strat

In [ ]:
strat_value = 100
total_fees = 0
stored_values = [strat_value]
weights_dict = {}
fees=0.0000

# Premier rebalancement
df_subset = factor_df.loc[(factor_df["Date"] == all_dates[0]), :]
initial_ptf = portfolio.construct_portfolio(
    df_subset,
    rebalance_weight=True
)
weights = dict(zip(initial_ptf['Ticker'], initial_ptf['Weight']))
weights_dict[all_dates[0]] = weights


rebalancing_dates = get_rebalancing_dates(all_dates, FrequencyType.MONTHLY)
tickers = price_df.columns
rebalancing_dt = []

for t in tqdm(range(1, len(all_dates)), desc=f"Running Backtesting", leave = False):

    returns_dict = returns_df.loc[returns_df['Date'] == all_dates[t], tickers].squeeze().to_dict()
    prev_weights = np.array([weights_dict[all_dates[t-1]][ticker] for ticker in tickers])

    daily_returns = np.array([returns_dict.get(ticker, 0) for ticker in tickers])
    return_strat = np.dot(prev_weights, daily_returns)
    new_strat_value = strat_value * (1 + return_strat)

    # Rebalancement selon le type spécifié
    if t in rebalancing_dates:

        rebalancing_dt.append(all_dates[t])
        df_subset = factor_df.loc[(factor_df["Date"] == all_dates[t]), :]
            
        # Construire le portefeuille avec les nouveaux poids
        df_ptf = portfolio.construct_portfolio(
            df_subset,
            rebalance_weight=True
        )
        new_weights = dict(zip(df_ptf['Ticker'], df_ptf['Weight']))
        transaction_costs = calculate_transaction_costs(weights, new_weights, fees)
        total_fees+=transaction_costs
        new_strat_value -= strat_value * transaction_costs
              
    else:
        new_weights = {ticker: weights[ticker] * (1 + returns_dict[ticker]) for ticker in weights}
        total_weight = sum(new_weights.values())
        new_weights = {ticker: weight / total_weight for ticker, weight in new_weights.items()}
    # Stockage des nouveaux poids et valeurs
    weights_dict[all_dates[t]] = new_weights
    stored_values.append(new_strat_value)

    weights = new_weights
    strat_value = new_strat_value
result = output(f"Momentum", stored_values, weights_dict, all_dates, rebalancing_dt, total_fees)

Running Backtesting:  10%|▉         | 324/3290 [00:00<00:02, 1057.36it/s]

In [41]:
print(result.df_statistics.head(10))

result.ptf_value_plot.show()
result.ptf_drawdown_plot.show() 
result.ptf_weights_plot

                Metrics Momentum
0          Total Return  233.25%
1     Annualized Return    9.66%
2            Volatility   16.02%
3          Sharpe Ratio     0.48
4         Sortino Ratio     0.61
5          Max Drawdown  -30.72%
6               VaR 95%   -1.59%
7              CVaR 95%   -2.43%
8  Frais de transaction    0.00%


In [53]:
stoxx = pd.read_excel("datas/stoxx600.xlsx")
stoxx = stoxx.set_index("Date")

In [50]:
data = pd.DataFrame(stored_values, index=all_dates, columns=["Momentum"])

In [54]:
df = data.merge(stoxx, left_index=True, right_index=True, how="left")

In [56]:
df['SXXP Index'] = df['SXXP Index'] / df['SXXP Index'].iloc[0] * 100

In [60]:
fig = go.Figure()

# Ajout des courbes
fig.add_trace(go.Scatter(x=df.index, y=df["Momentum"], mode='lines', name="Momentum"))
fig.add_trace(go.Scatter(x=df.index, y=df["SXXP Index"], mode='lines', name="SXXP Index", line=dict(dash="dash")))

# Personnalisation du graphique
fig.update_layout(title="Évolution de Momentum et SXXP Index",
                  xaxis_title="Date",
                  yaxis_title="Valeur",
                  template="plotly_white",
                  xaxis=dict(tickangle=-45))

# Affichage
fig.show()

In [61]:
def compute_drawdown(series):
    peak = series.cummax()  # Maximum atteint jusqu'à chaque point
    drawdown = (series - peak) / peak  # Drawdown en pourcentage
    return drawdown

df["Drawdown_Momentum"] = compute_drawdown(df["Momentum"])
df["Drawdown_SXXP"] = compute_drawdown(df["SXXP Index"])

# Création de la figure Plotly
fig = go.Figure()

# Ajout des courbes de drawdown
fig.add_trace(go.Scatter(x=df.index, y=df["Drawdown_Momentum"], 
                         mode='lines', name="Drawdown Momentum", 
                         line=dict(color='red')))
fig.add_trace(go.Scatter(x=df.index, y=df["Drawdown_SXXP"], 
                         mode='lines', name="Drawdown SXXP Index", 
                         line=dict(color='blue', dash="dash")))

# Personnalisation du graphique
fig.update_layout(title="Drawdowns de Momentum et SXXP Index",
                  xaxis_title="Date",
                  yaxis_title="Drawdown (%)",
                  template="plotly_white",
                  xaxis=dict(tickangle=-45),
                  yaxis=dict(tickformat=".2%"),  # Format en pourcentage
                  shapes=[dict(type="line", x0=df.index.min(), x1=df.index.max(), 
                               y0=0, y1=0, line=dict(color="black", width=1, dash="dot"))])  # Ligne à 0%

# Affichage
fig.show()
